In [4]:
import os
import gc
import pyperclip as clip
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *
import selenium.webdriver as webdriver

% reload_ext autoreload
% autoreload 2

In [5]:
PATH = os.path.abspath(r"C:\Users\rsilva\Desktop\SEI")
HOME = os.path.abspath("C:/Users/rsilva/gdrive/projects/programming/automation/")

In [ ]:
keys = ('Nome/Razão Social', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cep', 'Município', 'UF', 'Número Fistel')

In [6]:
def string_endereço(dados):
    
    d = {}
    
    s = 'A(o)<br>' 
    
    s += dados["Nome/Razão Social"].title()
    
    s += '<br>' + dados["Logradouro"].title() + ", " + dados["Número"] + " " 
    
    s += dados["Complemento"].title() + " " 
    
    s += dados["Bairro"].title() + '<br>' 
    
    s += "CEP: " + dados["Cep"] + " - " + dados["Município"].title() + " - " + dados["UF"] 
            
    s += "<br><br>" + "<b>FISTEL: " + dados["Número Fistel"] + "</b>"
    
    d["A"] = s
    
    return d

In [7]:
def lastRow(ws, col=2):
    """ Find the last row in the worksheet that contains data.

    idx: Specifies the worksheet to select. Starts counting from zero.

    workbook: Specifies the workbook

    col: The column in which to look for the last cell containing data.
    """

    #ws = workbook.sheets[idx]

    lwr_r_cell = ws.cells.last_cell      # lower right cell
    lwr_row = lwr_r_cell.row             # row of the lower right cell
    lwr_cell = ws.range((lwr_row, col))  # change to your specified column

    if lwr_cell.value is None:
        lwr_cell = lwr_cell.end('up')    # go up untill you hit a non-empty cell

    return lwr_cell.row

In [233]:
cpf = '03606367481'

In [234]:
proc = r'53504.010787/2018-41'

# SEI - Manipulação de Processo

## Autenticação

In [213]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [214]:
sei_ = sei.login_sei(browser, usr, pwd)

In [215]:
sei_ = sei.Sei(sei_)

In [216]:
sei_.itera_processos()

A página demorou muito tempo para carregar ou há somente 1 página de Processos


## Doc Externo

In [ ]:
boleto = sistemas.Boleto(driver)

boleto.imprime_boleto(cpf, 'id_cpf')

In [244]:
p = sei_.go_to_processo(proc)

In [245]:
files = sorted(os.listdir(PATH))

In [246]:
file = os.path.join(PATH, files[-1])

In [220]:
p.incluir_doc_externo('Boleto', file, arvore="Estação Fixa")

## Enviar Ofício

In [ ]:
p = sei_.go_to_processo(proc)

In [ ]:
p.incluir_oficio('RC_OF_Sem_Contexto_PPDESS', string_endereço(dados))

## Enviar Email

In [221]:
p = sei_.go_to_processo(proc)

In [222]:
email = 'gilmar-santos444@hotmail.com'

assunto = 'Serviço Rádio do Cidadão - Boleto Estação Fixa - Processo: {0} - ANATEL/RSAB'.format(proc)

msg = 'E-mail_PPDESS'

p.send_doc_por_email('Boleto', (email, assunto, msg))

## Controles

In [ ]:
for proc, dados in sei_._processos.items():
    if dados['atribuicao'] == 'deleicfaria':
        p = sei_.go_to_processo(proc)
        y = input()
        if y == 'y':
            p.edita_marcador()
            #p.edita_postit()
            #p.excluir_acomp_especial()
            p.concluir_processo()             

In [ ]:
p = sei_.go_to_processo(proc)

In [231]:
p.edita_marcador(tipo="RC - Boleto Enviado", content='Até 03/11/2018')

In [ ]:
p.edita_postit("Analisado - Controlar pagamento e Emitir Licença", True)

In [ ]:
p = sei_.go_to_processo(proc)

In [ ]:
p.edita_marcador()

In [230]:
p.edita_postit()

In [ ]:
p.excluir_acomp_especial()

In [ ]:
p.concluir_processo()

# Scpx - Manipulação de Cadastro

In [10]:
driver = webdriver.Ie()
scpx = sistemas.Scpx(driver)

## Consulta

In [12]:
scpx.consulta(cpf, timeout=2)

## Serviço

In [236]:
scpx.servico_incluir(cpf, num_processo=proc, silent=True, timeout=2)

In [ ]:
#scpx.servico_excluir(cpf, documento="2724547")
scpx.imprime_consulta(cpf)

## Estação

In [ ]:
scpx.prorrogar_rf(cpf)

In [ ]:
scpx.prorrogar_estacao(cpf)

In [237]:
scpx.incluir_estacao(cpf, 'Fixa', 'PX2I1980', sequencial='001', timeout=5)

TimeoutException: Message: 


In [ ]:
scpx.incluir_estacao(cpf, 'Fixa', 'PX2C3116', sequencial='001', timeout=5)

In [239]:
scpx.movimento_transferir(cpf, 'B', 'E', proc=proc, timeout=1)

Informe o campo: Movimento Atual
Operação realizada com sucesso!


In [ ]:
scpx.movimento_transferir(cpf, 'A', 'G', proc=proc)

In [ ]:
scpx.movimento_cancelar(cpf)

In [240]:
scpx.licenciar_estacao(cpf, silent=True, timeout=1)

Caso clique em "OK" será gerado o boleto do PPDESS (autorização do serviço). 
Utilize novamente a opção ESTAÇÃO > LICENCIAR para gerar os demais boletos, do PPDUR e TFI.


In [242]:
scpx.licenciar_estacao(cpf, ppdess=False, silent=True, timeout=2)

Para esta operação haverá cobrança de taxa para cada estação, conforme abaixo.

 Total de estações:
 Fixa: 1
 
 Estações Fixas: 7241 - Preço Público pelo Direito de Uso  de Radiofrequência (PPDUR): R$20,00 (único e se devido)
 Estações Fixas: 8766 - Taxa de Fiscalização de Instalação - TFI: R$33,52

 Data da validade da licença: 

Deseja continuar?


TimeoutException: Message: 


In [243]:
dados = scpx.extrai_cadastro(cpf, timeout=1)

Não há mais de um registro de Outorga


## Licença

In [ ]:
scpx.imprimir_licenca(cpf)

# Sigec

In [ ]:
sigec = sistemas.Sigec(driver)

In [ ]:
sigec.consulta_geral(cpf)

In [ ]:
dados.values()

# Sec

In [ ]:
import pandas as pd

df = pd.read_excel("files/registros_atualizados.xlsx", dtype=str, na_values='nan').fillna("")

In [ ]:
df['Ano do Óbito'] = df["Ano do Óbito"].str.replace('nan', "")

df['Complemento'] = df["Complemento"].str.replace('nan', "")

df["Data de Nascimento"] = df["Data de Nascimento"].str.replace("-", '')

df["Complemento"] = df["Complemento"].str.replace('nan', "")

df['Cidade'] = df["Cidade"].apply(lambda x: str(x).split("/")[0])

In [ ]:
df = df.iloc[:8335]
df["Erros"] = ""

In [ ]:
total = df.shape[0]

In [ ]:
driver = webdriver.Firefox()
sec = sistemas.Sec(driver, 'rsilva', "Savorthemom3nts")

In [ ]:
for i in range(10,100):
    
    try:
        sec.atualiza_cadastro(df.iloc[i])
        
    except:
        
        df.loc[i, "Erros"] = True
        
        next
        
        gc.collect()   
    

# SCRA

In [ ]:
scra = sistemas.Scra(scra.driver)

In [ ]:
scra.consulta(cpf)

In [ ]:
scra.imprimir_licenca(cpf)

In [ ]:
dados = scra.extrai_cadastro(cpf)

# Testes

In [ ]:
tipo = "Outorga: Rádio do Cidadão"
atribuicao = 'rodrigoca'

In [ ]:
processos = [p for p in sei_.get_processos().values() if p['atribuicao'] == 'rodrigoca']

In [ ]:
mm = [p for p in processos if p['tipo'] == tipo]

In [ ]:
for p in mm:
    print(p.get('marcador'))

In [ ]:
concluir = list()

In [ ]:
for processo in processos:
    
    if 'Licença' in str(processo.get('marcador')):
        
        p = sei_.go_to_processo(processo['numero'])
        
        i = input()
        
        if i == '1': 
            
            concluir.append(processo)
            
            
        gc.collect()

In [ ]:
for processo in concluir:
    
    p = sei_.go_to_processo(processo['numero'])
    
    
    sleep(2)
    
    try:    
        p.edita_marcador()    
    
        p.edita_postit()
    
        p.excluir_acomp_especial()
        
        p.concluir_processo() 
            
    except:
            
            next

In [ ]:
#dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)


In [ ]:
for num, dado in tupla[30:]:
        
    p = sei_.go_to_processo(num)
    
    i = input()
    
    if i =='y':
        
        try:
            
            dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)
                               
            p.incluir_oficio('RC_OF_Sem_Contexto_Licença', string_endereço(dados))
            
            p.edita_marcador(tipo='RC - Aguardando AR/Resposta', content='Até 31/07/2018')        

            
        except:
            
            next            
            
    if i == 'c':
        
        p.edita_marcador()
        p.edita_postit()
        p.excluir_acomp_especial()
        p.concluir_processo() 

## Outros

In [ ]:
for prova, gabarito in zip(files, scaneados):

    nome = prova.split(".")[0]
    
    p.incluir_doc_externo("Prova", os.path.join(SCAN, gabarito), arvore=nome)
    
    sleep(5)
    

In [ ]:
from bs4 import BeautifulSoup as soup
html = soup(scpx.driver.page_source, 'lxml')

In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for td in tr.find_all('td', string=True):
        key = td.text.strip(" :")
        value = td.find_next_sibling('td')            
        if value and key not in dict_ and value.text.strip() != "":
            dict_[key] = value.text.strip()
            print("{0} {1}".format(key, value.text.strip())) 


In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for label in tr.find_all('label', string=True):
        print("{0} {1}".format(label.attrs.get('id'), label.text.strip()))                           

In [ ]:
from bs4 import NavigableString
def next_is_string(tag):
    return isinstance(tag.next_element, NavigableString)             

# Boleto

In [8]:
import xlwings as xw

sht = xw.Book(os.path.join(HOME,'files\pagamentos.xlsm')).sheets("Consulta")

In [9]:
row = 5
cpf = str(sht.cells(row, 6).value)[:11]
proc = sht.cells(row, 4).value
last_row = lastRow(sht)

In [228]:
last_row = lastRow(sht)

In [229]:
for i in range(1, 4):

    sht.cells(last_row+i, 2).value = dados['Boleto PPDESS'][:11]
    sht.cells(last_row+i, 3).value = i
    sht.cells(last_row+i, 4).value = proc    

In [198]:
for num, p in sei_.get_processos().items():
    
    try:
    
        if p['atribuicao'] == 'rsilva':

            p = sei_.go_to_processo(num)

            tree = p.armazena_arvore()

            for k, v in tree.items():

                if "Licença" in k:                

                    input()
                    
    except NoAlertPresentException:
        
        next               
                
    
    